In [1]:
%matplotlib inline

In [2]:
import os
import imp
import bcolz as bc
import numpy as np

In [3]:
import keras
from keras.preprocessing.image import ImageDataGenerator as IDG

from keras.layers import Input, Dense, Flatten, Conv2D, BatchNormalization, AveragePooling2D, ELU, GaussianDropout, Add, GlobalAveragePooling2D, Activation
from keras.applications.xception import Xception
from keras.models import Model, load_model
from keras import backend as K


Using TensorFlow backend.


In [4]:
from config import config,bcolz_array_iterator
imp.reload(config)
imp.reload(bcolz_array_iterator)
origin_dir = config.origin_dir
group_dir = config.group_dir
bcolz_dir = config.bcolz_dir
config_dir = config.config_dir

BcolzArrayIterator = bcolz_array_iterator.BcolzArrayIterator

In [5]:
data = bc.carray(rootdir=os.path.join(bcolz_dir,"data"),mode="r")
labels = bc.carray(rootdir=os.path.join(bcolz_dir,"labels"),mode="r")
data_valid = bc.carray(rootdir=os.path.join(bcolz_dir,"data_valid"),mode="r")
labels_valid = bc.carray(rootdir=os.path.join(bcolz_dir,"labels_valid"),mode="r")

In [ ]:
data,labels

from matplotlib import pyplot as plt

a=data[12458][:]
a=np.array(a,dtype=np.uint8)
plt.figure(figsize=(10,10))
plt.imshow(a)

# buildmodel

In [7]:
def conv_block(inx,iter_num):
    x = BatchNormalization()(inx)
    x = Conv2D(64,(3,3),padding="same")(x)
    x = AveragePooling2D()(x)
    x = GaussianDropout(0.2)(x)
    x = ELU()(x)
    
    return x
    

In [8]:
model_path = os.path.join(config_dir,"model.h5")
try:
    model = load_model(model_path)
except:
    input_layer = Input((256,256,3))
    #x = Xception(input_shape=(256,256,3),include_top=False,weights=None,pooling="avg")(input_layer)
    first_layer = Conv2D(10,(1,1),padding="same")(input_layer)  #split channel

    x = first_layer
    for i in range(2):
        x = conv_block(x,i)

    x = Conv2D(193,(3,3),padding="same")(x)
    x = GlobalAveragePooling2D()(x)
    
    output_layer = Activation("softmax")(x)

    model = Model(input_layer,outputs=output_layer)

    model.compile("adam","categorical_crossentropy",metrics=["acc"])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 10)      40        
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 10)      40        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      5824      
_________________________________________________________________
average_pooling2d_1 (Average (None, 128, 128, 64)      0         
_________________________________________________________________
gaussian_dropout_1 (Gaussian (None, 128, 128, 64)      0         
_________________________________________________________________
elu_1 (ELU)                  (None, 128, 128, 64)      0         
__________

In [10]:
batch_size = 32

In [11]:
batch_train = BcolzArrayIterator(data,labels,batch_size=batch_size)
batch_valid = BcolzArrayIterator(data_valid,labels_valid,batch_size=batch_size)

In [12]:
model.fit_generator(batch_train,(len(data)/batch_size),validation_data=batch_valid,validation_steps=len(data_valid),epochs=6)

Epoch 1/6
999/998 [==============================] - 572s - loss: 3.7310 - acc: 0.2187 - val_loss: 3.1278 - val_acc: 0.3177
Epoch 2/6
999/998 [==============================] - 560s - loss: 2.4404 - acc: 0.4676 - val_loss: 2.1917 - val_acc: 0.5081
Epoch 3/6
999/998 [==============================] - 559s - loss: 1.8462 - acc: 0.5963 - val_loss: 1.8186 - val_acc: 0.5858
Epoch 4/6
999/998 [==============================] - 561s - loss: 1.5304 - acc: 0.6618 - val_loss: 1.5218 - val_acc: 0.6229
Epoch 5/6
999/998 [==============================] - 562s - loss: 1.3256 - acc: 0.7020 - val_loss: 1.2847 - val_acc: 0.6792
Epoch 6/6
999/998 [==============================] - 567s - loss: 1.1786 - acc: 0.7308 - val_loss: 1.3657 - val_acc: 0.6760


In [13]:
model.save(model_path)